To be up-to-date on the most current version of this code. Check out our GitHub repository: https://github.com/Neatherblok/Facial_Emotion_Detec

In [1]:
#!pip install datasets
#!pip install --upgrade --force-reinstall huggingface_hub


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/ad/30/2281c062222dc39328843bd1ddd30ff3005ef8e30b2fd09c4d2792766061/fsspec-2024.2.0-py3-none-any.whl.metadata
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\rdede\\Documents\\GitHub\\Facial_Emotion_Detec\\.env\\Lib\\site-packages\\~aml\\_yaml.cp310-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Preparation

In [31]:
import torch
from torchvision import transforms
from torchvision.transforms import Lambda, Resize
from datasets import load_dataset

''' 
Creates a class object MyDataset
************
__init__
  This function will load a part of the ImageNet-1K dataset specified in the 
  class caller.
  Options are train, validation, and test.
************
__getitem__
  This function will return two variables containing each a variable present 
  in the original ImageNet-1K dataset.
  Before returning these variables, it will split up and transform the image 
  into a fixed resolution of 256 by 256 pixels.
  
Returns: data, label
************
__len__
Returns: the length of the loaded dataset.
'''

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        # Loads the dataset that needs to be transformed
        self.dataset = load_dataset("imagefolder", data_dir=f"Data_(FER2013)", split=f"{dataset}")        

    def __getitem__(self, idx):
        # Sample row idx from the loaded dataset
        sample = self.dataset[idx]
        
        # Split up the sample example into an image and label variable
        data, label = sample['image'], sample['label']
        
        transform = transforms.Compose([
            transforms.Resize((48, 48)),  # Resize to size 48x48
            Lambda(lambda x: x.convert("L") if x.mode != "L" else x),  # Convert all images to grayscale format
            transforms.ToTensor(),  # Transform image to Tensor object
            transforms.Normalize(mean=[0.485],
                                 std=[0.2295])
        ])
        
        return transform(data), torch.tensor(label)

    def __len__(self):
        return len(self.dataset)

In [32]:
# Load and execute transformations on datasets
train_set = MyDataset('train')
val_set = MyDataset('test')

Resolving data files:   0%|          | 0/28709 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/7178 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28709 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/7178 [00:00<?, ?it/s]

In [33]:
train_set[0]

(tensor([[[-1.2589, -1.5665, -1.8570,  ...,  0.1593,  0.4669, -0.6438],
          [-1.1393, -1.5323, -1.7374,  ...,  0.2448,  0.4669, -0.5925],
          [-1.0710, -1.6007, -1.7032,  ...,  0.3131,  0.4327, -0.5925],
          ...,
          [-0.3533, -0.4045, -0.4045,  ...,  0.4327, -0.3362, -0.6609],
          [-0.2849, -0.2166, -0.1824,  ...,  0.4669, -0.0628, -0.6438],
          [-0.3362, -0.3362, -0.1995,  ...,  0.3302,  0.2106, -0.6950]]]),
 tensor(0))